# FERC to FERC Match

A classifier for identifying FERC plant time series in FERC Form 1 data.

Input: FERC plant record  
Output: The ID of the group of records that it ought to be part of

Training data: A list of lists of unique FERC plant record IDs (each record ID is the concatenation of: report year, respondent id, supplement number, and row number). It could also be stored as a dataframe where each column is associated with a year of data(some of which could be empty). Not sure what the best structure would be.

In [ ]:
%load_ext autoreload
%autoreload 3

In [ ]:
import faiss
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, Normalizer, OneHotEncoder
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA

from pudl.etl import defs
from ferc1_eia_match.name_cleaner import CompanyNameCleaner
from ferc1_eia_match.candidate_set_creation import DataframeEmbedder
from ferc1_eia_match import config
from ferc1_eia_match.metrics import ferc_to_ferc

## Get input data

In [ ]:
# use denorm_plants_all_ferc1 instead?
plants_df = defs.load_asset_value("denorm_plants_steam_ferc1")

In [ ]:
fuel_df = defs.load_asset_value("denorm_fuel_by_plant_ferc1")

In [ ]:
fuel_cols = list(fuel_df.filter(regex=".*_fraction_mmbtu$").columns)

In [ ]:
plants_df = plants_df.merge(
    fuel_df[["utility_id_ferc1", "plant_name_ferc1", "report_year"] + fuel_cols],
    on=["utility_id_ferc1", "plant_name_ferc1", "report_year"],
    how="left",
)

In [ ]:
utility_name_cleaner = CompanyNameCleaner()
# default rules except keep words in parentheses
plant_name_cleaner = CompanyNameCleaner(
    cleaning_rules_list=[
        "replace_amperstand_between_space_by_AND",
        "replace_hyphen_between_spaces_by_single_space",
        "replace_underscore_by_space",
        "replace_underscore_between_spaces_by_single_space",
        "remove_text_puctuation_except_dot",
        "remove_math_symbols",
        "add_space_before_opening_parentheses",
        "add_space_after_closing_parentheses",
        "remove_parentheses",
        "remove_brackets",
        "remove_curly_brackets",
        "enforce_single_space_between_words",
    ]
)

In [ ]:
plants_df = (
    plants_df.pipe(
        utility_name_cleaner.get_clean_df,
        "utility_name_ferc1",
        "utility_name",
    )
    .pipe(plant_name_cleaner.get_clean_df, "plant_name_ferc1", "plant_name")
)

Probably should adapt the DataframeEmbedder class to function with a "pipeline"-like structure for all these embedding functions.

Vectorize FERC data:  
* plant_name (via TF-IDF, with ngram_min and ngram_max as parameters)
* plant_type (OneHot encoded categorical feature)
* construction_type (OneHot encoded categorical feature)
* capacity_mw (MinMax scaled numerical feature)
* construction year (OneHot encoded categorical feature)
* utility_id_ferc1 (OneHot encoded categorical feature)
* fuel_fraction_mmbtu (several MinMax scaled numerical columns, which are
normalized and treated as a single feature.)

Note: Integer nulls are filled with 0 in the PUDL module

In [ ]:
ngram_min=2
ngram_max=10
plant_name_ferc1_wt=2.0
plant_type_wt=2.0
construction_type_wt=1.0
capacity_mw_wt=1.0
construction_year_wt=1.0
utility_id_ferc1_wt=1.0
fuel_fraction_wt=1.0

In [ ]:
str_cols = ["plant_type", "construction_type"]
num_cols = ["capacity_mw", "construction_year", "utility_id_ferc1"] + fuel_cols

In [ ]:
plants_df[str_cols] = plants_df[str_cols].astype(str).fillna("").astype("category")

In [ ]:
plants_df[num_cols] = plants_df[num_cols].fillna(0)

In [ ]:
embedder = ColumnTransformer(
    transformers=[
        (
            "plant_name_ferc1",
            TfidfVectorizer(
                analyzer="char", ngram_range=(ngram_min, ngram_max)
            ),
            "plant_name_ferc1",
        ),
        (
            "plant_type",
            OneHotEncoder(categories="auto"),
            ["plant_type"],
        ),
        (
            "construction_type",
            OneHotEncoder(categories="auto"),
            ["construction_type"],
        ),
        ("capacity_mw", MinMaxScaler(), ["capacity_mw"]),
        (
            "construction_year",
            OneHotEncoder(categories="auto"),
            ["construction_year"],
        ),
        (
            "utility_id_ferc1",
            OneHotEncoder(categories="auto"),
            ["utility_id_ferc1"],
        ),
        (
            "fuel_fraction_mmbtu",
            Pipeline(
                [("scaler", MinMaxScaler()), ("norm", Normalizer())]
            ),
            fuel_cols,
        ),
    ],
    transformer_weights={
        "plant_name_ferc1": plant_name_ferc1_wt,
        "plant_type": plant_type_wt,
        "construction_type": construction_type_wt,
        "capacity_mw": capacity_mw_wt,
        "construction_year": construction_year_wt,
        "utility_id_ferc1": utility_id_ferc1_wt,
        "fuel_fraction_mmbtu": fuel_fraction_wt,
    },
)

In [ ]:
plants_matrix_sparse = embedder.fit_transform(plants_df)

In [ ]:
plants_matrix_sparse.shape

In [ ]:
%%time
d = plants_matrix_sparse.shape[1]
plants_matrix = np.float32(plants_matrix_sparse.todense())

In [ ]:
plants_small = plants_matrix[:2000]

In [ ]:
plants_small_df = plants_df[:2000].copy()


In [ ]:
d = 500

In [ ]:
%%time
pca_small = PCA(n_components=d)
plants_small_pca = pca_small.fit_transform(np.asarray(plants_small))

In [ ]:
%%time
pca = PCA(n_components=d)
plants_matrix_pca = pca.fit_transform(np.asarray(plants_matrix))

In [ ]:
dist_matrix_small = pairwise_distances(plants_small_pca, metric='euclidean')

In [ ]:
dist_matrix = pairwise_distances(plants_matrix_pca, metric='euclidean')

In [ ]:
dist_matrix_small.shape, dist_matrix.shape

In [ ]:
dist_matrix_small.mean(), dist_matrix_small.max(), dist_matrix.mean(), dist_matrix.max()

In [ ]:
# TODO: would be nice to get rid of the for loop
def penalize_same_report_year(df, dist_matrix, penalty=10):
    report_years = range(df.report_year.min(), df.report_year.max()+1)
    for yr in report_years:
        # get the indices of all the record pairs that have matching report years
        yr_idx = df[df.report_year == yr].index
        yr_match_pairs_idx = np.array(np.meshgrid(yr_idx, yr_idx)).T.reshape(-1, 2)
        mat = np.full(dist_matrix.shape, 0)
        idx_x = yr_match_pairs_idx[:, 0]
        idx_y = yr_match_pairs_idx[:, 1]
        mat[idx_x, idx_y] = penalty
        # distance from node to itself should still be 0
        np.fill_diagonal(mat, 0)
        dist_matrix += mat
    return dist_matrix

In [ ]:
dist_matrix_w_penalty_small = penalize_same_report_year(plants_small_df, dist_matrix_small.copy())

In [ ]:
dist_matrix_w_penalty = penalize_same_report_year(plants_df, dist_matrix.copy())

In [ ]:
# similarity threshold of .75 is used in the PUDL module, but we need a distance threshold
d_threshold = .5

https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_clustering_metrics.html#sphx-glr-auto-examples-cluster-plot-agglomerative-clustering-metrics-py

Maybe cosine distance isn't the best metric here even though it's used in the PUDL module

What are we optimizing for? 
- largest possible clusters so there are few straggler records
- no overlapping report years within a cluster

Options for ensuring report year doesn't overlap:
- calculate a distance matrix, for all record pairs with the same report year make the distance bigger. Make metric = 'precomputed' for the model and pass distance metric into clustering model
- add a dimension to the post-PCA vector? not sure how this would work without training data

In [ ]:
%%time
# use the distance matrix and pass it into sklearn agglomerative clustering
agg_clustering_small = AgglomerativeClustering(n_clusters=None,
                                         metric="precomputed",
                                         linkage="average",
                                         distance_threshold=d_threshold,
                                         compute_distances=True)
labels_small = agg_clustering_small.fit_predict(np.asarray(dist_matrix_w_penalty))

In [ ]:
agg_clustering_small.distances_.mean()

In [ ]:
plants_small_df["id"] = labels_small
ferc_to_ferc.measure_matching(plants_small_df, d_threshold, run_tags={"dataset": "small"})

In [ ]:
# do records with the same record year end up in the same group?
# is there a way to build this restriction into the classifier?
# could add a "distance to the rest of the cluster" metric and choose the record with smaller distance
plants_small_df.groupby(by=["id", "report_year"]).size().value_counts().sort_index()


In [ ]:
plants_small_df.groupby(by=["id"]).size().hist(bins=20)

In [ ]:
plants_small_df.groupby(by=["id"]).size().mean()

In [ ]:
plants_small_df.groupby("id").plant_id_ferc1.nunique().value_counts().sort_index()

In [ ]:
%%time
# use the distance matrix to pass into agglomerative clustering
agg_clustering = AgglomerativeClustering(n_clusters=None,
                                         metric="precomputed",
                                         linkage="average",
                                         distance_threshold=d_threshold,
                                         compute_distances=True)
labels = agg_clustering.fit_predict(np.asarray(dist_matrix_w_penalty))

In [ ]:
plants_labels_df = plants_df.copy()

In [ ]:
plants_labels_df["id"] = labels
ferc_to_ferc.measure_matching(plants_small_df, d_threshold, run_tags={"dataset": "full"})

In [ ]:
# do records with the same record year end up in the same group?
# is there a way to build this restriction into the classifier?
# could add a "distance to the rest of the cluster" metric and choose the record with smaller distance
plants_labels_df.groupby(by=["id", "report_year"]).size().value_counts()

In [ ]:
plants_labels_df.groupby(by=["id"]).size().hist(bins=20)

In [ ]:
plants_labels_df.groupby("id").size().mean()

In [ ]:
plants_labels_df.id.max()

In [ ]:
plants_labels_df.groupby("id").plant_id_ferc1.nunique().value_counts().sort_index().head(10)

What is the average distance between records in a group?

In [ ]:
labels.min(),labels.max()

Graph the results

Deal with orphaned records